In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master("local").getOrCreate()

In [4]:
df = spark.read.csv("titanic.csv",header=True,inferSchema=True)

In [5]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [6]:
df.show(n=10)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

# Target Variable: Survived

In [7]:
my_cols = df.select(['Survived',
 'Pclass','Sex',
 'Age',
 'SibSp',
 'Parch','Fare','Embarked'])

In [8]:
my_final_data = my_cols.na.drop()

In [9]:
my_final_data.show(n=10)

+--------+------+------+----+-----+-----+-------+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|
|       1|     1|female|38.0|    1|    0|71.2833|       C|
|       1|     3|female|26.0|    0|    0|  7.925|       S|
|       1|     1|female|35.0|    1|    0|   53.1|       S|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|
|       0|     1|  male|54.0|    0|    0|51.8625|       S|
|       0|     3|  male| 2.0|    3|    1| 21.075|       S|
|       1|     3|female|27.0|    0|    2|11.1333|       S|
|       1|     2|female|14.0|    1|    0|30.0708|       C|
|       1|     3|female| 4.0|    1|    1|   16.7|       S|
+--------+------+------+----+-----+-----+-------+--------+
only showing top 10 rows



In [10]:
from pyspark.ml.feature import (VectorAssembler, VectorIndexer,
                               OneHotEncoder, StringIndexer)

In [11]:
gender_indexer = StringIndexer(inputCol="Sex",outputCol="SexIndex")
gender_encoder = OneHotEncoder(inputCol = "SexIndex",outputCol="SexVec")

In [12]:
embarked_indexer = StringIndexer(inputCol="Embarked",outputCol="EmbarkedIndex")
embarked_encoder = OneHotEncoder(inputCol = "EmbarkedIndex",outputCol="EmbarkedVec")

In [13]:
assembler = VectorAssembler(inputCols=['Pclass','SexVec','EmbarkedVec','Age','SibSp','Parch','Fare'],outputCol='features')

In [14]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [15]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

In [16]:
pipeline = Pipeline(stages=[gender_indexer,embarked_indexer,
                           gender_encoder,embarked_encoder,
                           assembler,log_reg_titanic])

In [17]:
train_data, test_data = my_final_data.randomSplit([0.7,0.3])

In [18]:
fitted_model = pipeline.fit(train_data)

In [19]:
results = fitted_model.transform(test_data)

# Evaluation

In [34]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [35]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')

In [36]:
my_eval.evaluate(results) #returns AUC

0.7732883317261331

In [38]:
results.select('Survived','prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
+--------+----------+
only showing top 20 rows

